<h2>FINAL DATA PREPROCESSING & MODEL TRAINING</h2>

Script ini digunakan untuk mempreprocess data menjadi data final yang dapat digunakan untuk training model, dan training model LSTM itu sendiri. Secara umum, script ini meliputi:
<h4>Data Preprocessing</h4>
- Padding & Truncating data, pada dasarnya memastikan seluruh trj_id memiliki jumlah data yang sama, yakni 20 data, dengan menambahkan data yang kurang dengan 0 dan memotong data yang kelebihan dan mengambil 20 data paling belakang.
- Membagi data menjadi X (koordinat, keceptan, dll yang dimasukkan ke model untuk melatih model) dan y (koordinat yang benar untuk dibandingkan dengan koordinat hasil prediksi model). y akan diambil dari koordinat terakhir tiap trajectory.
- Membagi data menjadi training data dan testing data, dengan pembagian 90%/10%.
<br/>
<h4>Model Training</h4>
- Mendefinisikan dan melatih model LSTM berdasarkan data yang telah siap.
- Arsitektur, epoch, loss, optimizer, dan dll dapat dengan bebas diubah-ubah untuk mencoba model lain.
<br/>
Jika ingin melihat performa model pertama yang telah saya latih, maka model itu sudah disave dalam folder models dengan nama model_v1.keras. Model dapat ditest menggunakan script Model Test.ipynb

In [48]:
## Import libraries
import pandas as pd
import numpy as np
import tensorflow as tf
from sklearn.model_selection import train_test_split
import datetime

In [49]:
# Load Tensorboard
%load_ext tensorboard

The tensorboard extension is already loaded. To reload it, use:
  %reload_ext tensorboard


In [50]:
## Read data from csv
resampled_data = pd.read_csv('../clean_data.csv', index_col='trj_id').drop('Unnamed: 1', axis=1)
print(resampled_data)

          rawlat      rawlng      speed     bearing  hour_of_day  day_of_week
trj_id                                                                       
1      -6.198042  106.769008   4.322800  179.920000           14            3
1      -6.200972  106.769202   8.014167  173.233333           14            3
1      -6.205394  106.769768  10.116136  171.477273           14            3
1      -6.210496  106.771217   9.307667  156.683333           14            3
1      -6.214969  106.773830  10.103333  139.777778           14            3
...          ...         ...        ...         ...          ...          ...
9999   -6.187751  106.845707  10.584667  329.600000            4            6
9999   -6.184123  106.843546   4.508780  324.512195            4            6
9999   -6.182706  106.842869   2.776724  287.137931            4            6
9999   -6.180504  106.842337   5.244333  326.850000            4            6
9999   -6.179029  106.841998   2.330952  231.095238            4

In [51]:
# Pad and truncate the timestamps in the dataframe
# Ini buat kita samain input modelnya, gw potong timestampnya jadi pasti ada 20 timestamp per sample. Kalo lebih dipotong, kalo kurang ditambahin 0 di depannya
# Pad value None --> pake koordinat pertama

def pad_truncate_dataframe(df, max_len, padding='pre', truncating='post', pad_value=None):
  # Split the dataframe by samples (first level of multi-index)
  samples = df.groupby(level=0)

  # Define a function to pad/truncate a single sample
  def pad_truncate_sample(sample):
    # Extract the values from a single sample
    values = sample.values
    first_element = values[0]
    first_element[2] = 0
    first_element[3] = 0

    # Pad/truncate automatically using pad_sequences
    if pad_value == None:
      padded_truncated = tf.keras.preprocessing.sequence.pad_sequences(
          [values], maxlen=max_len, padding=padding, truncating=truncating, value=first_element, dtype='float64'
      )[0]
    else:
      padded_truncated = tf.keras.preprocessing.sequence.pad_sequences(
          [values], maxlen=max_len, padding=padding, truncating=truncating, value=pad_value, dtype='float64'
      )[0]

    # Convert back to pandas dataframe
    return pd.DataFrame(padded_truncated, columns=sample.columns)

  # Apply the function to each sample and recreate the multi-index dataframe
  padded_df = samples.apply(pad_truncate_sample)
  return padded_df

df_new = pad_truncate_dataframe(resampled_data, 61)

In [52]:
## Convert the multi index dataframe to a numpy 3D array for better integration to TensorFlow (samples, timesteps, features)
numpy_data = df_new.to_xarray().to_array().to_numpy()
numpy_data = np.transpose(numpy_data, (1, 2, 0))
numpy_data.shape # Should be (55994, 21, 6) if 55994 samples, 21 timesteps, and 6 features

(55994, 61, 6)

In [53]:
output_steps = 10

## Split the data to x (feature values) and y (target values)
simple_data = numpy_data[:, :, 0:2]
x_data = simple_data[:, :-output_steps, :]
y_data = simple_data[:, -10:, :]

In [54]:
# Split the data to train and test splits (train to train the model, test to test the model on new data after trained)
test_size = 0.1

x_train, x_test, y_train, y_test = train_test_split(x_data, y_data, test_size=test_size)

In [55]:
## Define normalization layer
normalize_layer = tf.keras.layers.Normalization(axis=-1, input_shape=(50, 2))
normalize_layer.adapt(simple_data)

c:\Users\adrie\AppData\Local\Programs\Python\Python39\lib\site-packages\keras\src\layers\preprocessing\normalization.py:99: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


In [56]:
## Get Mean and Variance of Normalization layer to use for normalizing y and denormalizing later
normalize_weights = normalize_layer.get_weights()

mean_variance = np.array([normalize_weights[0][0:2], normalize_weights[1][0:2]])

np.save('../mean_variance_D.npy', mean_variance)

In [57]:
## Normalize y
normalize_y = tf.keras.layers.Normalization(mean=mean_variance[0], variance=mean_variance[1])

y_train = normalize_y(y_train)
y_test = normalize_y(y_test)
print(y_train)

tf.Tensor(
[[[ 0.13437289 -1.1068265 ]
  [ 0.09475513 -1.1150434 ]
  [ 0.06204621 -1.1287383 ]
  ...
  [-0.09997586 -1.0946767 ]
  [-0.12975013 -1.0851254 ]
  [-0.1497591  -1.0729756 ]]

 [[-0.10605215  1.7926235 ]
  [-0.08135602  1.7751362 ]
  [-0.06568922  1.734754  ]
  ...
  [ 0.03453042  1.5796858 ]
  [ 0.06551903  1.5593894 ]
  [ 0.13284577  1.5634627 ]]

 [[ 0.68540156  0.14123274]
  [ 0.67989105  0.14179459]
  [ 0.67186904  0.12304315]
  ...
  [ 0.6236773   0.11433461]
  [ 0.6242523   0.11307047]
  [ 0.6247905   0.11770567]]

 ...

 [[ 0.86214286  0.13420974]
  [ 0.8880763   0.12971501]
  [ 0.89409745  0.14270759]
  ...
  [ 0.9959684   0.34806043]
  [ 1.008006    0.41161868]
  [ 1.0150344   0.44884062]]

 [[-1.1760768   0.66978455]
  [-1.2858547   0.66936314]
  [-1.3997954   0.67729914]
  ...
  [-1.9528801   0.6015911 ]
  [-2.0544384   0.55250025]
  [-2.1276343   0.517315  ]]

 [[ 0.35759482 -0.04487704]
  [ 0.32176265 -0.08469751]
  [ 0.29584298 -0.11594991]
  ...
  [ 0.2281068

In [58]:
## Save test data for model testing
np.save('../x_test_D.npy', x_test)
np.save('../y_test_D.npy', y_test)

In [59]:
num_features = 2

# Clear any previous models
tf.keras.backend.clear_session()

# Define the model
model = tf.keras.Sequential()
model.add(normalize_layer)
model.add(tf.keras.layers.LSTM(128, return_sequences=True))
model.add(tf.keras.layers.LSTM(128))
model.add(tf.keras.layers.Dense(num_features * output_steps))
model.add(tf.keras.layers.Reshape([output_steps, num_features]))


# Define optimizer
lr = 1e-3
opt = tf.keras.optimizers.Adam(learning_rate=lr)

model.compile(loss='huber', optimizer=opt, metrics=['mae', 'mse'])
model.summary()

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ normalization (Normalization)   │ (None, 50, 2)          │             5 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lstm (LSTM)                     │ (None, 50, 128)        │        67,072 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lstm_1 (LSTM)                   │ (None, 128)            │       131,584 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 20)             │         2,580 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ reshape (Reshape)               │ (None, 10, 2)          │             0 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 201,241 (786.10 KB)

 Trainable params: 201,236 (786.08 KB)

 Non-trainable params: 5 (24.00 B)

In [60]:
## Define LR scheduling (optional if want to use or not)
start_lr = lr
min_lr = 0.00001
max_lr = 0.001
rampup_epochs = 0
sustain_epochs = 0
exp_decay = 0.0

# Define the scheduling function
def schedule(epoch):
  def lr(epoch, start_lr, min_lr, max_lr, rampup_epochs, sustain_epochs, exp_decay):
    if epoch < rampup_epochs:
      lr = (max_lr - start_lr)/rampup_epochs * epoch + start_lr
    elif epoch < rampup_epochs + sustain_epochs:
      lr = max_lr
    else:
      lr = (max_lr - min_lr) * exp_decay**(epoch-rampup_epochs-sustain_epochs) + min_lr
    return lr
  return lr(epoch, start_lr, min_lr, max_lr, rampup_epochs, sustain_epochs, exp_decay)

In [61]:
## Define callbacks and fit the model
log_dir = "../logs/fit/" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=log_dir, histogram_freq=1)

early_stopping = tf.keras.callbacks.EarlyStopping(monitor='val_loss', mode='min', patience=20)

lr_scheduler = tf.keras.callbacks.LearningRateScheduler(schedule)

model.fit(
  x=x_train, 
  y=y_train,
  epochs=200,
  validation_data=(x_test, y_test),
  callbacks=[tensorboard_callback, early_stopping,]) #Can add lr_scheduler

model.evaluate(x_test, y_test)

Epoch 1/200
1575/1575 ━━━━━━━━━━━━━━━━━━━━ 116s 72ms/step - loss: 0.0310 - mae: 0.1540 - mse: 0.0698 - val_loss: 0.0075 - val_mae: 0.0861 - val_mse: 0.0151
Epoch 2/200
1575/1575 ━━━━━━━━━━━━━━━━━━━━ 112s 71ms/step - loss: 0.0073 - mae: 0.0834 - mse: 0.0146 - val_loss: 0.0066 - val_mae: 0.0771 - val_mse: 0.0132
Epoch 3/200
1575/1575 ━━━━━━━━━━━━━━━━━━━━ 96s 61ms/step - loss: 0.0063 - mae: 0.0760 - mse: 0.0126 - val_loss: 0.0057 - val_mae: 0.0718 - val_mse: 0.0115
Epoch 4/200
1575/1575 ━━━━━━━━━━━━━━━━━━━━ 99s 63ms/step - loss: 0.0058 - mae: 0.0728 - mse: 0.0117 - val_loss: 0.0056 - val_mae: 0.0705 - val_mse: 0.0112
Epoch 5/200
1575/1575 ━━━━━━━━━━━━━━━━━━━━ 117s 74ms/step - loss: 0.0058 - mae: 0.0730 - mse: 0.0117 - val_loss: 0.0058 - val_mae: 0.0727 - val_mse: 0.0115
Epoch 6/200
1575/1575 ━━━━━━━━━━━━━━━━━━━━ 103s 65ms/step - loss: 0.0058 - mae: 0.0721 - mse: 0.0115 - val_loss: 0.0055 - val_mae: 0.0710 - val_mse: 0.0111
Epoch 7/200
1575/1575 ━━━━━━━━━━━━━━━━━━━━ 110s 70ms/step - loss: 

[0.0050555807538330555, 0.06622148305177689, 0.010111422277987003]

In [62]:
## Save the model
model.save('../models/model_v3.keras')